In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf

import matplotlib as mpl
mpl.use('TkAgg')
from matplotlib import pyplot as plt

hidden_size = 30
num_layers = 2

time_steps = 10
training_steps = 100
batch_size = 32

training_examples = 10000
testing_examples = 1000
# 采样间隔
sample_gap = 0.01


def generate_data(seq):
    """用sin函数前面的time_steps个点的值，预测第i + time_steps点的函数值"""
    X = []
    y = []
    for i in range(len(seq) - time_steps):
        X.append([seq[i: i + time_steps]])
        y.append([seq[i + time_steps]])
    # x->[None, 1, time_steps] y->[None, 1]
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)


def lstm_model(X, y, is_training):
    """lstm_model 模型"""
    cell = tf.nn.rnn_cell.MultiRNNCell([
        tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
        for _ in range(num_layers)
    ])
    
    # X->[batch_size, 1, time_steps] ,  outputs->[batch_size, 1, hidden_size]
    outputs, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    
    # [batch_size, hidden_size]
    output = outputs[:, -1, :]
    
    # [batch_size, 1]
    predictions = tf.contrib.layers.fully_connected(
        output, 1, activation_fn=None
    )

    if not is_training:
        return predictions, None, None

    loss = tf.losses.mean_squared_error(labels=y, predictions=predictions)

    # 优化
    train_op = tf.contrib.layers.optimize_loss(
        loss,
        tf.train.get_global_step(),
        optimizer="Adagrad",
        learning_rate=0.1
    )

    return predictions, loss, train_op


def train(sess, train_X, train_y):
    ds = tf.data.Dataset.from_tensor_slices((train_X, train_y))
    ds = ds.repeat().shuffle(1000).batch(batch_size)
    X, y = ds.make_one_shot_iterator().get_next()

    # 创建模型
    with tf.variable_scope("model"):
        predictions, loss, train_op = lstm_model(X, y, True)

    sess.run(tf.global_variables_initializer())
    for i in range(training_steps):
        _, l = sess.run([train_op, loss])
        if i % 100 == 0:
            print("train step: " + str(i) + ", loss: " + str(l))


def run_eval(sess, test_X, test_y):
    ds = tf.data.Dataset.from_tensor_slices((test_X, test_y))
    ds = ds.batch(1)
    X, y = ds.make_one_shot_iterator().get_next()

    with tf.variable_scope("model", reuse=True):
        prediction, _, _ = lstm_model(X, [0.0], False)
    predictions = []
    labels = []
    for i in range(testing_examples):
        p, l = sess.run([prediction, y])
        predictions.append(p)
        labels.append(l)

    predictions = np.array(predictions).squeeze()
    print(predictions)
    labels = np.array(labels).squeeze()
    rmse = np.sqrt(((predictions - labels) ** 2).mean(axis=0))
    print("Mean Square Error is : %f" % rmse)

    plt.figure()
    plt.plot(predictions, label='predictions')
    plt.plot(labels, label='labels')
    plt.legend()
    plt.show()

# 生成数据 
test_start = (training_examples + time_steps) * sample_gap
test_end = test_start + (testing_examples + time_steps) * sample_gap
train_X, train_y = generate_data(np.sin(np.linspace(
    0, test_start, training_examples+time_steps, dtype=np.float32)))
test_X, test_y = generate_data(np.sin(np.linspace(
    test_start, test_end, testing_examples + time_steps, dtype=np.float32)))

with tf.Session() as sess:
    train(sess, train_X, train_y)
    run_eval(sess, test_X, test_y)


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
train step: 0, loss: 0.43974516
[-4.62413073e-01 -4.52627003e-01 -4.42692399e-01 -4.32609260e-01
 -4.22378540e-01 -4.12000954e-01 -4.01477337e-01 -3.90808165e-01
 -3.79995346e-01 -3.69040787e-01 -3.57945740e-01 -3.46712768e-01
 -3.35343719e-01 -3.23840678e-01 -3.12206268e-01 -3.00443649e-01
 -2.88555324e-01 -2.76545703e-01 -2.64417231e-01 -2.52173543e-01
 -2.39818588e-01 -2.27356479e-01 -2.14791253e-01 -2.02127665e-01
 -1.89370260e-01 -1.76523998e-01 -1.63593933e-01 -1.50585324e-01
 -1.37503609e-01 -1.24354392e-01 -1.11143410e-01 -9.78765637e-02
 -8.45598429e-02 -7.11994395e-02 -5.78015372e-02 -4.43725437e-02
 -3.09188291e-02 -1.74468830e-02 -3.96323577e-03  9.52557102e-03
  2.30129473e-02  3.64923254e-02  4.99571450e-02  6.34008795e-02
  7.68170282e-02  9.0

In [ ]:
import numpy as np
x = np.array([[
    [23,23,23,23]
],[
    [23,23,23,23]
]])
y = x[:, -1, :]
print(x.shape)
print(x)
print(y.shape)
print(y)